In [1]:
import numpy as np
import pandas as pd
import time

# read in transformed data
d = pd.read_csv("temp.csv")
d.head()

,PD1,GzmB,CD8a,CD103,CD56,CD25,CD4,CD19,CD49a,CD3,FoxP3
0,1.760079,2.241769,0.447316,0.951417,1.085694,0.365136,1.159030,0.744424,1.756490,1.208541,0.897291
1,2.504053,1.743894,0.725944,1.063522,1.146853,0.924040,2.429543,1.594169,3.008561,1.925828,0.740108
2,1.171469,1.729824,0.942350,0.670537,1.141923,0.644870,0.998817,0.800615,1.419161,1.141207,0.937181
3,1.464891,1.908043,0.499899,0.927789,0.677614,1.049692,0.864599,0.746302,1.478173,1.268048,0.886642
4,1.989203,1.786417,0.857825,0.784642,0.617290,1.169930,0.363880,0.945732,1.475951,1.183369,0.803843


In [34]:
from sklearn.covariance import graphical_lasso
import time

def gaussian(X, mu, cov):
    # calculates gaussian likelihood for all rows of X
    [n,d] = X.shape
    p = np.zeros(n)
    X = X - mu
    
    for j in range(n):
        p[j] = np.exp(-0.5*np.dot(X[j,], np.dot(np.linalg.inv(cov),X[j,])) )/(2*np.pi)**(d/2)/np.linalg.det(cov)**0.5
    
    return p


# following along https://pdfs.semanticscholar.org/f1c3/e45b78d5b7614586ae86074cbbc90b061635.pdf
def sgmm(X, k, lamda=0.001, iters = 100):
    [n,p] = X.shape
    
    # random initialization
    mu = 4*np.random.rand(k,p)      # all component means
    cov = np.zeros((k,p,p))       # all component cov matrices
    for j in range(k):
        temp = np.random.rand(p)-0.5
        cov[j,:,:] = np.outer(temp,temp)
        np.fill_diagonal(cov[j,:,:],1)
    pi = np.random.rand(k)         # all component priors
    w = np.zeros((n,k))             # membership weight matrix
    
    obj = np.zeros((iters,k))
    start = time.time()
    
    for cnt in range(iters):
        # each iteration
        for j in range(k):     # for each cluster
            # 1. compute the required variables
            w[:,j] = pi[j]*gaussian(X,mu[j,],cov[j,:,:])
            mu[j,] = np.sum(w[:,j]*X.T,axis=1)/np.sum(w[:,j])
            S = np.dot(w[:,j]*(X-mu[j,]).T,X-mu[j,])/np.sum(w[:,j])         # sample covariance
            # 2. run graphical lasso w coordinate descent to find covariance matrix that maximizes penalized log-likelihood
            # https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3019769/pdf/kxm045.pdf
            print(cnt*k+j+1)
            [cov[j,:,:],_,costs] = graphical_lasso(S, lamda, cov_init = S + lamda*np.eye(p), mode='cd', max_iter=200, return_costs=True)
            obj[cnt,j] = costs[-1][0]
            # obj[cnt,j] = np.log(np.linalg.det(C)) - np.matrix.trace(np.dot(S,C)) - lamda*np.sum(np.abs(C))
        # 3. update priors
        pi = np.mean(w, axis=0)
        
    print('Total time: {} s'.format(time.time()-start))
    return (mu, cov, p, obj)

In [35]:
temp = sgmm(X=d.to_numpy(), k=8, lamda=0.001, iters = 10)

1
2
3
4
5
6
7


C:\Users\adity\Anaconda3\lib\site-packages\sklearn\covariance\graph_lasso_.py:265: ConvergenceWarning: graphical_lasso: did not converge after 200 iteration: dual gap: 1.236e-03
  % (max_iter, d_gap), ConvergenceWarning)


8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


C:\Users\adity\Anaconda3\lib\site-packages\sklearn\covariance\graph_lasso_.py:265: ConvergenceWarning: graphical_lasso: did not converge after 200 iteration: dual gap: 7.869e-04
  % (max_iter, d_gap), ConvergenceWarning)


25
26


C:\Users\adity\Anaconda3\lib\site-packages\sklearn\covariance\graph_lasso_.py:265: ConvergenceWarning: graphical_lasso: did not converge after 200 iteration: dual gap: 2.329e-02
  % (max_iter, d_gap), ConvergenceWarning)


27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
Total time: 1211.6683311462402 s


In [14]:
temp[1][4,:,:]

array([[ 0.14040843,  0.06451202,  0.09836349,  0.0560468 ,  0.15734877,
         0.15510414, -0.00980028,  0.12763507,  0.07923221,  0.06867938,
         0.1668774 ],
       [ 0.06451202,  0.07115515,  0.04464197,  0.04977936,  0.08442175,
         0.0812483 ,  0.0265278 ,  0.06761845,  0.0771438 ,  0.03549034,
         0.08265843],
       [ 0.09836349,  0.04464197,  0.1488592 ,  0.03910967,  0.11626024,
         0.11552657, -0.02395498,  0.09606145,  0.05124505,  0.05451044,
         0.13073584],
       [ 0.0560468 ,  0.04977936,  0.03910967,  0.08800881,  0.07235671,
         0.0690452 ,  0.02543954,  0.05809679,  0.07026565,  0.03427382,
         0.0713678 ],
       [ 0.15734877,  0.08442175,  0.11626024,  0.07235671,  0.21184333,
         0.19459721, -0.0055321 ,  0.15811935,  0.10540786,  0.08153451,
         0.2034541 ],
       [ 0.15510414,  0.0812483 ,  0.11552657,  0.0690452 ,  0.19459721,
         0.20931182, -0.01065074,  0.15715098,  0.09896116,  0.08035927,
         0.201

In [23]:
X = np.random.randint(0,4,(4,4))
mu = np.random.randint(0,4,(4,))
print(X)
print(mu)
print(np.mean(X,axis=0))

[[1 3 1 1]
 [3 0 0 1]
 [1 1 1 0]
 [1 0 1 0]]
[1 1 0 2]
[1.5  1.   0.75 0.5 ]


In [39]:
import pickle
with open('ninjaboi','wb') as f:
    pickle.dump(temp,f)

In [41]:
with open('ninjaboi','rb') as f:
    temp = pickle.load(f)